In [46]:
import scipy.optimize as so
import numpy as np
import h5py as h5
    

In [47]:
def UFid(U,V,Dim):
#    return np.sum(abs(U-V)**2)
    return abs(np.trace(np.matmul(U.conj().T,V))/Dim)**2

In [48]:
def QuantumFourier(Dim): 
    pi=np.arccos(-1.)
    x=np.array(range(Dim))
    return np.exp( 1j*2*pi*np.reshape( np.kron(x,x), [Dim,Dim] )/Dim )/np.sqrt(Dim)

In [49]:
def RandomUnitary(Dim):
    return np.linalg.qr( np.random.randn(Dim,Dim)+1j*np.random.randn(Dim,Dim) )[0]

In [50]:
def UFunction(Phase,N,F,Dim):
    U=np.eye(Dim)
    for k in range(N):
        U= np.exp( 1j*np.append([0] ,Phase[k*(Dim-1):(k+1)*(Dim-1)] ) )[:,np.newaxis]*U
        U=np.matmul(F,U)
    return np.exp(1j*np.append( [0], Phase[N*(Dim-1):(N+1)*Dim-1]))[:,np.newaxis]*U

In [51]:
def PUMIOpt(F,Dim,N,U):
    Seed = np.ones( (N+1)*(Dim-1) )
    TargetOpt = lambda x: 1-UFid( U , UFunction( x , N , F , Dim ) , Dim )
    argsmin = {"method": "L-BFGS-B"}
    Res=so.basinhopping(TargetOpt , Seed,minimizer_kwargs=argsmin,niter=5*10**2,disp=True)
    return Res.fun

In [52]:
def main():
    Repe=100
    Dim=4
    N=Dim
    
    F = QuantumFourier(Dim)
    file=h5.File("FFdataDim=4.h5","r") # Cambiar Nombre archivo 
    UU=file["re"][:,:,:].T+1j*file["im"][:,:,:].T
    
    ind2=file["Index"][:]-1
    MC=len(ind2)
    file.close()
    Data=np.zeros([MC,Repe])
    l=np.where(Data[:,0]<1)
    print(Dim,"-----",MC)
    for j in range(Repe):
        for i in range(MC):
            Data[i,j]=PUMIOpt( F,Dim,N,UU[:,:,ind2[ l[0][i] ]] )
        l2=np.where(Data[l,j]<10**(-10))
        for h in range(np.size(l2[0])):
            Data[l2[0][h],:]=Data[l2[0][h],j]
        l=np.where(Data[:,j]>10**(-10))
        MC=np.size(l[0])
        print(j,"-----",np.size(l[0]))

    file=h5.File("FFdataDim=4py.h5","w") #Cambiar nombre al archivo de salida
    file["infi"]=Data
    file["index1"]=l[0]
    file["index2"]=l2[0]
    file.close()

In [53]:
from scipy.io import loadmat

In [54]:
data = loadmat('data_contraejemplos.mat')['data_contraejemplos']

In [55]:
infids_comparison = np.zeros((8,2))

for Dim in range(3,11):

    L = 0
    infids = data[0,Dim-3][0,1][:,0]

    idx_worst   = np.argmax( infids )
    infid_worst = infids[idx_worst] 

    N=Dim
    F=QuantumFourier(Dim)

    u = data[0,Dim-3][0,0][:,:,idx_worst]

    infid_BH = PUMIOpt( F,Dim,N,u )

    infids_comparison[Dim-3,:] = [infid_worst,infid_BH]

basinhopping step 0: f 0.0496994
basinhopping step 1: f 0.0496994 trial_f 0.0496994 accepted 1  lowest_f 0.0496994
basinhopping step 2: f 0.0496994 trial_f 0.0496994 accepted 1  lowest_f 0.0496994
basinhopping step 3: f 0.0496994 trial_f 0.0496994 accepted 1  lowest_f 0.0496994
basinhopping step 4: f 0.0496994 trial_f 0.0496994 accepted 1  lowest_f 0.0496994
basinhopping step 5: f 0.0496994 trial_f 0.0496994 accepted 1  lowest_f 0.0496994
basinhopping step 6: f 0.0496994 trial_f 0.0496994 accepted 1  lowest_f 0.0496994
basinhopping step 7: f 0.0496994 trial_f 0.0496994 accepted 1  lowest_f 0.0496994
found new global minimum on step 7 with function value 0.0496994
basinhopping step 8: f 0.0496994 trial_f 0.0496994 accepted 1  lowest_f 0.0496994
basinhopping step 9: f 0.0496994 trial_f 0.0496994 accepted 1  lowest_f 0.0496994
basinhopping step 10: f 0.0496994 trial_f 0.0496994 accepted 1  lowest_f 0.0496994
basinhopping step 11: f 0.0496994 trial_f 0.0496994 accepted 1  lowest_f 0.049699

In [56]:
infids_comparison

array([[4.96993884e-02, 4.96993884e-02],
       [1.05638681e-01, 1.05638681e-01],
       [3.53096077e-03, 3.53096284e-03],
       [1.67199208e-05, 4.30217082e-06],
       [5.94534671e-06, 8.49747111e-07],
       [5.38061842e-06, 9.37286069e-08],
       [1.92266611e-06, 4.16724656e-06],
       [9.37497460e-07, 3.78641204e-05]])

In [57]:
np.save( 'julia_vs_python', infids_comparison )